## 1. Last inn data:

In [20]:
import geopandas as gpd
import time
import os
import psutil
import pandas as pd
from shapely import wkt
from shapely import wkb

## Utfør partisjonering
Merk at du må slette `data` mappa om du har kjørt u1_ eller u2_ og opprette ny, samt legge inn filene i `data/raw` på nytt!

In [23]:
DATA_DIR = "AIS/Data"
PROCESSED_DIR = "AIS/Data/processed"

CENTER_POINT_KRS = (8.0182, 58.1599) # (longitude, latitude)
STD_BUFFER_DISTANCE = 500 #meter
STD_RADIUS = 1000 #meter
STD_CRS = 'EPSG:4326'

TIME_COLUMN = 'date_time_utc'
STD_PARTITION_COLUMNS = ['date_time_utc', 'ship_type']
SHIP_DISPLAY_COLUMNS = ['mmsi', 'ship_name', 'ship_type', 'longitude', 'latitude']

NUM_OF_SHIPS = 10
MAX_NUM_OF_SHIPS = 100

def create_folders(DATA_DIR):
    """
    Oppretter nødvendig mappestruktur for konvertering.

    Args:
        data_folder: Sti til hovedmappen for data

    Returns:
        dict: Stier til opprettede mapper
    """
    # Definer mappestruktur
    raw_folder = os.path.join(DATA_DIR, "raw")
    processed_folder = os.path.join(DATA_DIR, "processed")
    source_folder = os.path.join(DATA_DIR, "source")
    incoming_folder = os.path.join(DATA_DIR, "incoming")

    # Opprett mapper
    os.makedirs(DATA_DIR, exist_ok=True)
    os.makedirs(raw_folder, exist_ok=True)
    os.makedirs(processed_folder, exist_ok=True)
    os.makedirs(source_folder, exist_ok=True)
    os.makedirs(incoming_folder, exist_ok=True)

    print("Mappestruktur opprettet.")

    # Returner stier for senere bruk
    return {
        "data_folder": os.path.abspath(DATA_DIR),
        "raw_folder": os.path.abspath(raw_folder),
        "processed_folder": os.path.abspath(processed_folder),
        "source_folder": os.path.abspath(source_folder),
        "incoming_folder": os.path.abspath(incoming_folder)
    }

def add_time_partitioning_columns(df, time_column=TIME_COLUMN):
    """
    Legger til kolonner for tidspartisjonering (kun time for partisjonering, men beholder dato-kolonner).
    """
    if time_column not in df.columns:
        print(f"Advarsel: Tidsstempelkolonne '{time_column}' finnes ikke")
        return df

    if not pd.api.types.is_datetime64_any_dtype(df[time_column]):
        print(f"Advarsel: Kolonnen '{time_column}' er ikke en datetime-kolonne")
        return df

    df_with_time = df.copy()
    df_with_time['year'] = df_with_time[time_column].dt.year
    df_with_time['month'] = df_with_time[time_column].dt.month
    df_with_time['day'] = df_with_time[time_column].dt.day
    df_with_time['hour'] = df_with_time[time_column].dt.hour

    return df_with_time

def create_geodataframe(df):
    """
    Oppretter en GeoDataFrame fra en DataFrame ved å finne koordinater eller geometrikolonner.
    """

    # Sjekk for lat/long kolonner
    if 'longitude' in df.columns and 'latitude' in df.columns:
        try:
            return gpd.GeoDataFrame(
                df,
                geometry=gpd.points_from_xy(df.longitude, df.latitude),
                crs=STD_CRS
            )
        except Exception as e:
            print(f"Kunne ikke opprette geometri fra lat/long: {e}")

    # Sjekk for andre geometrikolonner
    geom_columns = [col for col in df.columns if any(
        term in col.lower() for term in ['geom', 'coord', 'point', 'polygon', 'linestring', 'wkt']
    )]

    for col in geom_columns:
        if df[col].dtype != 'object':
            continue

        try:
            geom = df[col].apply(wkt.loads)
            return gpd.GeoDataFrame(df, geometry=geom, crs=STD_CRS)
        except Exception:
            continue

    return None

def save_partitioned_geoparquet(gdf, output_path, partition_columns):
    """
    Lagrer en GeoDataFrame som partisjonert GeoParquet.
    """
    # Konverterer geometri til WKB for å kunne partisjonere
    df_med_wkb = gdf.copy()
    df_med_wkb['geometry_wkb'] = df_med_wkb['geometry'].apply(lambda geom: wkb.dumps(geom))
    df_for_partisjon = df_med_wkb.drop(columns=['geometry'])
    # Utfør partisjonering
    df_for_partisjon.to_parquet(output_path, partition_cols=partition_columns)
    # Konverter hver partisjonert fil tilbake til GeoParquet
    convert_partitioned_files_to_geoparquet(output_path)
    return output_path

def create_year_folder(year, output_path):
    """Oppretter en mappe for et spesifikt år."""
    year_folder = os.path.join(output_path, f"year={year}")
    os.makedirs(year_folder, exist_ok=True)
    return year_folder

def prepare_wkb_dataframe(year_df):
    """Forbereder en dataframe med WKB-konvertert geometri for partisjonering."""
    # Konverter geometri til WKB for å kunne partisjonere
    year_df_wkb = year_df.copy()
    year_df_wkb['geometry_wkb'] = year_df_wkb['geometry'].apply(lambda geom: wkb.dumps(geom))
    year_df_wkb = year_df_wkb.drop(columns=['geometry'])
    return year_df_wkb

def convert_file_to_geoparquet(src_file, dst_file):
    """Konverterer en enkelt fil fra WKB-format til GeoParquet."""
    try:
        part_df = pd.read_parquet(src_file)
        if 'geometry_wkb' not in part_df.columns:
            return False

        part_df['geometry'] = part_df['geometry_wkb'].apply(lambda x: wkb.loads(x))
        part_df = part_df.drop(columns=['geometry_wkb'])
        part_gdf = gpd.GeoDataFrame(part_df, geometry='geometry', crs=STD_CRS)

        # Sørg for at målmappen eksisterer
        os.makedirs(os.path.dirname(dst_file), exist_ok=True)
        part_gdf.to_parquet(dst_file)
        return True
    except Exception as e:
        print(f"Advarsel: Kunne ikke konvertere {src_file} til GeoParquet: {e}")
        return False

def copy_and_convert_files(temp_year_folder, year_folder):
    """Kopierer og konverterer filer fra temp-mappen til målmappen."""
    success_count = 0
    error_count = 0

    # Opprett først alle mappene
    for root, dirs, _ in os.walk(temp_year_folder):
        for directory in dirs:
            src_dir = os.path.join(root, directory)
            rel_path = os.path.relpath(src_dir, temp_year_folder)
            dst_dir = os.path.join(year_folder, rel_path)
            os.makedirs(dst_dir, exist_ok=True)

    # Kopier og konverter filene
    for root, _, files in os.walk(temp_year_folder):
        for file in files:
            src_file = os.path.join(root, file)
            rel_path = os.path.relpath(src_file, temp_year_folder)
            dst_file = os.path.join(year_folder, rel_path)

            if convert_file_to_geoparquet(src_file, dst_file):
                success_count += 1
            else:
                error_count += 1

    return success_count, error_count

def convert_partitioned_files_to_geoparquet(PROCESSED_DIR):
    """
    Konverterer alle partisjonerte parquet-filer til GeoParquet format.
    """
    error_count = 0

    for root, _, files in os.walk(PROCESSED_DIR):
        for file in files:
            if not file.endswith('.parquet'):
                continue

            parquet_path = os.path.join(root, file)
            try:
                # Les dataframe
                part_df = pd.read_parquet(parquet_path)

                # Hopp over hvis den ikke har geometry_wkb
                if 'geometry_wkb' not in part_df.columns:
                    continue

                # Konverter WKB tilbake til geometri
                part_df['geometry'] = part_df['geometry_wkb'].apply(lambda x: wkb.loads(x))
                part_df = part_df.drop(columns=['geometry_wkb'])

                # Lag GeoDataFrame
                part_gdf = gpd.GeoDataFrame(part_df, geometry='geometry', crs=STD_CRS)

                # Skriv GeoParquet-filen
                part_gdf.to_parquet(parquet_path)
            except Exception as e:
                print(f"Feil ved konvertering av {parquet_path}: {e}")
                error_count += 1

    if error_count > 0:
        print(f"Advarsel: {error_count} filer kunne ikke konverteres til GeoParquet")

def convert_parquet_to_geoparquet(file_path, output_path, partition_columns=None):
    """
    Konverterer parquet-fil til GeoParquet-format med partisjonering.
    Args:
    file_path: Sti til parquet-filen
    output_path: Sti hvor GeoParquet-filen skal lagres
    partition_columns: Liste av kolonnenavn som skal brukes for partisjonering
    """
    try:
        df = pd.read_parquet(file_path)
    except Exception as e:
        print(f"Kunne ikke lese parquet-fil: {e}")
        return False

    # Håndter tidspartisjonering
    time_partitioning = False
    if partition_columns and TIME_COLUMN in partition_columns:
        partition_columns.remove(TIME_COLUMN)
        time_partitioning = True

    # Legg til alle tidspartisjoneringskolonner, men vi vil bare partisjonere på time
    if time_partitioning:
        df = add_time_partitioning_columns(df)
        # Sett opp partisjonering kun på 'hour'
        partition_columns = ['hour'] + (partition_columns or [])

    # Opprett GeoDataFrame
    gdf = create_geodataframe(df)
    if gdf is None:
        return False

    # Sjekk at alle partisjoneringskolonner finnes
    if partition_columns and not all(col in gdf.columns for col in partition_columns):
        print(f"Advarsel: Ikke alle partisjoneringskolonner finnes i datasettet")
        missing = [col for col in partition_columns if col not in gdf.columns]
        print(f"Manglende kolonner: {missing}")
        return False

    # For partisjonering, må vi sikre at output_path er en mappe
    if partition_columns:
        # Sjekk om output_path slutter med .parquet
        if output_path.endswith('.parquet'):
            # Lag en mappe i stedet for en fil
            directory_path = output_path
            # Sikre at mappen eksisterer
            os.makedirs(os.path.dirname(directory_path), exist_ok=True)
        else:
            directory_path = output_path

        # Utfør partisjonering
        return save_partitioned_geoparquet(gdf, directory_path, partition_columns)
    else:
        # Uten partisjonering, lagre direkte som fil
        gdf.to_parquet(output_path)
        return output_path

def convert_all_parquet_files(DATA_DIR, partition_columns=None):
    """
    Konverterer alle parquet-filer i en mappe til GeoParquet.
    """
    raw_folder = os.path.join(DATA_DIR, "raw")
    processed_folder = os.path.join(DATA_DIR, "processed")

    results = {
        "converted": [],
        "failed": []
    }

    # Finn alle parquet-filer
    parquet_files = [f for f in os.listdir(raw_folder)
                    if f.lower().endswith('.parquet') and os.path.isfile(os.path.join(raw_folder, f))]

    if not parquet_files:
        print("Ingen parquet-filer funnet i råmappen")
        return results

    # Konverter hver fil
    for filename in parquet_files:
        file_path = os.path.join(raw_folder, filename)
        base_filename = os.path.splitext(filename)[0]

        # For partisjonering, lag en mappe i stedet for en fil
        if partition_columns:
            output_dir = os.path.join(processed_folder, base_filename)
            os.makedirs(output_dir, exist_ok=True)
            output_path = output_dir
        else:
            # For ingen partisjonering, bruk fil-path
            output_path = os.path.join(processed_folder, f"{base_filename}.parquet")

        # Konverter filen
        resultat = convert_parquet_to_geoparquet(
            file_path,
            output_path,
            partition_columns.copy() if partition_columns else None,
        )

        if resultat:
            results["converted"].append(file_path)
        else:
            results["failed"].append(file_path)
            print(f"Kunne ikke konvertere: {file_path}")

    return results

def start_conversion_with_metrics(DATA_DIR, partition_columns=None):
    start_time = time.time()

    print(f"Starter konvertering og partisjonering av filer ...\n")
    print(f"Valgte kolonner: {partition_columns}")
    print("Vent litt...")

    #Utfør konverteringen
    results = convert_all_parquet_files(DATA_DIR, partition_columns)

    end_time = time.time()
    total_time = end_time - start_time

    print("\nKonvertering og partisjonering utført:")
    print(f"- Total behandlingstid: {total_time:.2f} sekunder")
    print(f"- Konverterte filer: {len(results['converted'])}")

    # Mål filstørrelse av partisjonerte filer
    total_partitioned_size_mb = 0
    processed_folder = os.path.join(DATA_DIR, "processed")
    for root, _, files in os.walk(processed_folder):
        for file in files:
            if file.endswith('.parquet'):
                file_path = os.path.join(root, file)
                total_partitioned_size_mb += os.path.getsize(file_path) / (1024 * 1024)

    return {
        "conversion_time": total_time,
        "partitioned_filesize_mb": total_partitioned_size_mb
    }

def get_total_raw_files_size(DATA_DIR):
    raw_folder = os.path.join(DATA_DIR, "raw")
    total_raw_size_mb = 0
    for file in os.listdir(raw_folder):
        file_path = os.path.join(raw_folder, file)
        if os.path.isfile(file_path):
            total_raw_size_mb += os.path.getsize(file_path) / (1024 * 1024)
    return total_raw_size_mb

def get_memory_usage_mb():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 * 1024)

In [24]:
create_folders(DATA_DIR);

Mappestruktur opprettet.


# 3. Kjør konvertering med og uten partisjonering

In [25]:
# Mål original filstørrelse
original_filesize_mb = get_total_raw_files_size(DATA_DIR)

# Uten partisjonering
memory_before_no_partition = get_memory_usage_mb()
results_no_partition = start_conversion_with_metrics(DATA_DIR, partition_columns=None)
memory_after_no_partition = get_memory_usage_mb()

# Med partisjonering
memory_before_partition = get_memory_usage_mb()
results_partition = start_conversion_with_metrics(DATA_DIR, partition_columns=STD_PARTITION_COLUMNS)
memory_after_partition = get_memory_usage_mb()

Starter konvertering og partisjonering av filer ...

Valgte kolonner: None
Vent litt...

Konvertering og partisjonering utført:
- Total behandlingstid: 0.38 sekunder
- Konverterte filer: 3
Starter konvertering og partisjonering av filer ...

Valgte kolonner: ['date_time_utc', 'ship_type']
Vent litt...

Konvertering og partisjonering utført:
- Total behandlingstid: 2.64 sekunder
- Konverterte filer: 3


# 4. Hvis resulater

In [26]:
data = {
    "Metode": ["Uten Partisjonering", "Med Partisjonering"],
    "Konverteringstid (sekunder)": [results_no_partition["conversion_time"], results_partition["conversion_time"]],
    "Filstørrelse etter konvertering (MB)": [original_filesize_mb, results_partition["partitioned_filesize_mb"]],
    "Minnebruk før konvertering (MB)": [memory_before_no_partition, memory_before_partition],
    "Minnebruk etter konvertering (MB)": [memory_after_no_partition, memory_after_partition],
    "Minnebruk økning (MB)": [memory_after_no_partition - memory_before_no_partition, memory_after_partition - memory_before_partition]

}

result_df = pd.DataFrame(data)
print(result_df)

                Metode  Konverteringstid (sekunder)  \
0  Uten Partisjonering                     0.381436   
1   Med Partisjonering                     2.635363   

   Filstørrelse etter konvertering (MB)  Minnebruk før konvertering (MB)  \
0                              5.785349                        464.28125   
1                             14.667778                        492.50000   

   Minnebruk etter konvertering (MB)  Minnebruk økning (MB)  
0                          492.50000               28.21875  
1                          599.59375              107.09375  
